<a href="https://colab.research.google.com/github/gmargoth/secmca_data/blob/main/secmca_data_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Run All

In [74]:
!git clone https://github.com/gmargoth/secmca_data
%cd secmca_data/
from secmca_data_downloader import secmca_download_csv

Cloning into 'secmca_data'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 24 (delta 6), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (24/24), 8.38 KiB | 8.38 MiB/s, done.
Resolving deltas: 100% (6/6), done.
/content/secmca_data/secmca_data


In [75]:
#ejemplo
serie = "TCN"
start_year, end_year = 1990, 2023
country_list =  ["Costa Rica", "El Salvador","Guatemala","Honduras","Nicaragua","República Dominicana","Panamá"]
df = secmca_download_csv(serie, start_year, end_year, country_list)

,El Salvador,Honduras,República Dominicana,Panamá,Guatemala,Costa Rica,Nicaragua
1990-Enero,6.64,3.40,6.33,1.0,3.59,84.19,NaN
1990-Febrero,6.90,3.45,6.33,1.0,3.81,84.68,NaN
1990-Marzo,6.90,4.00,6.33,1.0,3.92,85.41,NaN
1990-Abril,6.90,4.00,7.01,1.0,4.21,86.15,NaN
1990-Mayo,6.90,4.20,7.30,1.0,4.25,87.50,NaN
...,...,...,...,...,...,...,...
2023-Mayo,8.75,24.56,54.38,1.0,7.79,538.02,36.23
2023-Junio,8.75,24.59,54.77,1.0,7.82,539.89,36.26
2023-Julio,8.75,24.59,55.74,1.0,7.83,540.62,NaN
2023-Agosto,8.75,24.59,56.50,1.0,7.84,536.56,NaN
